In [ ]:
# !pip install --upgrade pip
# !pip install scikit-learn
# !pip install matplotlib
# !pip install tqdm
# !pip install opencv-python
# !pip install dlib


import os
import cv2
import dlib
import numpy as np
from tqdm import tqdm

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


In [ ]:
# Load the face detector from dlib
detector = dlib.get_frontal_face_detector()
detector

In [ ]:
# Load the facial landmarks predictor from dlib
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')
predictor

In [ ]:
# Extract facial features for a single image
def extract_features(image_path):
    # <class 'numpy.ndarray'>
    if isinstance(image_path, np.ndarray):
        image = image_path
    else:
        # Load the image
        image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the grayscale image
    faces = detector(gray)
    # Initialize an array to store the facial features
    features = []
    # Iterate over detected faces
    for face in faces:
        # Get the facial landmarks
        landmarks = predictor(gray, face)
        # Initialize an array to store the landmarks
        landmarks_array = np.zeros((68, 2), dtype=np.float32)
        # Iterate over the facial landmarks
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            # Store the landmark coordinates in the array
            landmarks_array[n] = (x, y)
        # Append the landmarks array to the features array
        features.append(landmarks_array)
    # Return the extracted features
    return features

In [ ]:
# Path to the directory containing training images
train_dir = './dataset'

# List to store the extracted features and corresponding labels
X = []
y = []
images_with_face = []

# Iterate over the training images
for person_name in os.listdir(train_dir):
    person_dir = os.path.join(train_dir, person_name)
    if os.path.isdir(person_dir):
        # Iterate over the images of the current person
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            # Extract features from the image
            features = extract_features(image_path)
            if len(features):
                # Add the features to the list
                X.extend(features)
                # Add the label to the list
                y.extend([person_name] * len(features))
                # if there is a face detected in the image, print the image path
                images_with_face.append(image_path)


In [ ]:
len(X), len(y)

In [ ]:
# for the first person in the dataset, plot all the images with faces and their corresponding facial landmarks in a grid
person_name = os.listdir(train_dir)[0]
person_dir = os.path.join(train_dir, person_name)
fig = plt.figure(figsize=(20, 20))
# Number of columns and rows based on number of images
columns = 6
rows = 10

# Limit the number of images to be plotted based on the grid size
num_images = min(len(images_with_face), columns * rows)

for i in range(1, num_images + 1):
    image_path = images_with_face[i-1]  # Adjust the index to start from 0
    image = mpimg.imread(image_path)
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(img_gray)
    for face in faces:
        landmarks = predictor(img_gray, face)
        for n in range(68):
            Cx = landmarks.part(n).x
            Cy = landmarks.part(n).y
            cv2.circle(image, (Cx, Cy), 10, (0, 255, 255), -1)

    fig.add_subplot(rows, columns, i)
    plt.imshow(image)
plt.show()


In [ ]:
# create a chart to show the number of images with faces
# create a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
# plot the number of images with faces
ax1.bar(np.unique(y), [y.count(i) for i in np.unique(y)])
ax1.set_title('Number of images with faces')
ax1.set_xlabel('Person')
ax1.set_ylabel('Number of images')
# plot a random image with a face
ax2.imshow(mpimg.imread(images_with_face[np.random.randint(len(images_with_face))]))
ax2.set_title('Random image with a face')
ax2.axis('off')
plt.show()


In [ ]:
# Convert the feature list to a NumPy array
X = np.array(X)
# Flatten the 2D array of features to 1D
X = X.reshape(X.shape[0], -1)
# Convert the label list to a NumPy array
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier
svm.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = svm.predict(X_test)

In [ ]:
# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [26]:
def identify_face_picture(image_path):
    # Extract features from the test image
    image_features = extract_features(image_path)
    # Flatten the 2D array of features to 1D
    image_features = np.array(image_features).reshape(1, -1)
    predicted_label = None
    if image_features.shape[1]:
        # Extract the label of the test image
        predicted_label = svm.predict(image_features)[0]
    # Return the name of the predicted label
    return predicted_label

def identify_face_video(video_path):
    # Open the video file for reading
    video = cv2.VideoCapture(video_path)
    # List to store the identified people
    identified_people = []
    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_count = 0
    # Read the video frame by frame
    for _ in tqdm(range(total_frames)):
        ret, frame = video.read()
        if not ret:
            break
        if frame_count % 10 == 0:
            # Extract features from the current frame
            frame_features = extract_features(frame)
            # If any faces are detected in the frame
            if len(frame_features) > 0:
                # Flatten the 2D array of features to 1D
                frame_features = np.array(frame_features).reshape(len(frame_features), -1)
                # Predict the labels for the frame features
                predicted_labels = svm.predict(frame_features)
                # Add the predicted labels to the identified people list
                identified_people.extend(predicted_labels)
        frame_count += 1
    
    # Release the video capture object
    video.release()

    # unique identified people
    unique_identified_people = set(identified_people)
    # count of identified people
    data = [(i, identified_people.count(i)) for i in unique_identified_people]
    if not len(data):
        return 'unknown'
    # get highest count and the person
    max_count = max(data, key=lambda x: x[1])
    return max_count[0]  # person label


In [27]:
test_dir = './input/'
output_dir = './output'
images_dir = os.path.join(output_dir, 'images')
videos_dir = os.path.join(output_dir, 'videos')

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create the images directory if it doesn't exist
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Create the videos directory if it doesn't exist
if not os.path.exists(videos_dir):
    os.mkdir(videos_dir)

# Iterate over the files in the test directory and its subdirectories
for root, dirs, files in os.walk(test_dir):
    print("DIRECTORY:", root, dirs)
    for file_name in tqdm(files):
        try:
            file_path = os.path.join(root, file_name)
            # Check if the file is an image
            if file_name.endswith(('.jpg', '.jpeg', '.png')):
                # Process the image
                predicted_label = identify_face_picture(file_path)
                if predicted_label:
                    label_images_dir = os.path.join(images_dir, predicted_label)
                    if not os.path.exists(label_images_dir):
                        os.mkdir(label_images_dir)
                    os.rename(file_path, os.path.join(label_images_dir, file_name))
                else:
                    unknown_images_dir = os.path.join(images_dir, 'unknown')
                    if not os.path.exists(unknown_images_dir):
                        os.mkdir(unknown_images_dir)
                    os.rename(file_path, os.path.join(unknown_images_dir, file_name))

            # Check if the file is a video
            elif file_name.endswith(('.mp4')):
                # Process the video frames
                predicted_label = identify_face_video(file_path)
                if predicted_label:
                    label_videos_dir = os.path.join(videos_dir, predicted_label)
                    if not os.path.exists(label_videos_dir):
                        os.mkdir(label_videos_dir)
                    os.rename(file_path, os.path.join(label_videos_dir, file_name))
                else:
                    unknown_videos_dir = os.path.join(videos_dir, 'unknown')
                    if not os.path.exists(unknown_videos_dir):
                        os.mkdir(unknown_videos_dir)
                    os.rename(file_path, os.path.join(unknown_videos_dir, file_name))
        except Exception as e:
            # print("error: file_path", file_path, e)
            pass


DIRECTORY: ./input/ ['Gallery', 'GBWhatsApp_Images', 'GBWhatsApp_Video']


0it [00:00, ?it/s]


DIRECTORY: ./input/Gallery ['owner']


0it [00:00, ?it/s]


DIRECTORY: ./input/Gallery\owner ['Haseeb', 'Muhammad Mustafa']


...

100%|██████████| 6705/6705 [22:14<00:00,  5.02it/s]  


DIRECTORY: ./input/GBWhatsApp_Images\Private []


100%|██████████| 1/1 [00:00<?, ?it/s]


DIRECTORY: ./input/GBWhatsApp_Images\Sent []


 16%|█▋        | 561/3425 [01:50<27:36,  1.73it/s]